In [ ]:
import keras

# Now you can safely import other libraries after installation
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import os
import boto3
import cv2
import pvlib
from pvlib import location
import datetime
from datetime import datetime
import pytz
latitude = 46.518404934146574
longitude = 6.565174801663707
batiment = location.Location(latitude, longitude, tz='UTC', altitude=400, name='ELD')
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import math
import matplotlib.pyplot as plt
import bisect
from datetime import datetime, timedelta, timezone
import plotly.graph_objects as go
import h5py
from sklearn.linear_model import Ridge, ElasticNet, LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
import itertools
from tqdm import tqdm
import concurrent.futures
import random as python_random
from influxdb_client import InfluxDBClient
np.random.seed(12345)
python_random.seed(12345)
access_key = '***'
secret_key = '***'
bucket_id  = '***'
endpoint = '***'
# Initialize the S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    endpoint_url=endpoint
)


#the model was trained using 2 images, with 5 min difference


def apply_circular_mask(image):
    center=(640, 512)
    radius=535
    # Create a mask with the same shape as the image
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    # Draw a filled white circle on the mask
    cv2.circle(mask, center, radius, (255, 255, 255), -1)
    # Apply the mask to the image
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    return masked_image


def correct_fisheye_full_view(input_image):
    """
    Corrects fisheye distortion for a given image and preserves the whole image.
    """
    # Get the size of the input image
    height, width = input_image.shape[:2]
    # Define intrinsic camera matrix and distortion coefficients
    k = np.array([[437.051833868485, 0, 610.832839633424],
              [0, 442.486815359197, 496.451253984502],
              [0, 0, 1]])
    d = np.array([-0.146542838644292, 0.0114721530486779, 0, 0])

    # Generate the new camera matrix based on the desired output view
    new_camera_matrix, _ = cv2.getOptimalNewCameraMatrix(k, d, (width, height), 1, (width, height))

    # Undistort the image
    undistorted_image = cv2.undistort(input_image, k, d, None, new_camera_matrix)

    return undistorted_image


import boto3
import os
import concurrent.futures
from tqdm import tqdm
from datetime import datetime, timedelta
import pytz
import pandas as pd
from datetime import timedelta
import cv2


def download_and_process_image(key):
    try:
        # Descargar desde S3
        response = s3.get_object(Bucket=bucket_id, Key=key)
        img_data = response['Body'].read()
        img_array = np.frombuffer(img_data, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        # Procesamiento directo (sin copia)
        img = apply_circular_mask(img)
        img = correct_fisheye_full_view(img)
        img_resized = cv2.resize(img, (250, 250), interpolation=cv2.INTER_AREA)

        return key, img_resized

    except Exception as e:
        print(f"Error downloading/processing image {key}: {e}")
        return key, None

def get_latest_images_from_s3(num_previous_images=2, image_delta_t=15):
    global s3, bucket_id

    access_key = '***'
    secret_key = '***'
    bucket_id  = '***'
    endpoint = '***'
    folder_name = 'All_sky_camera/'
    header = 'mean'

    s3 = boto3.client(
        's3',
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key,
        endpoint_url=endpoint
    )

    # Obtener claves
    paginator = s3.get_paginator('list_objects_v2')
    all_images = []
    for page in paginator.paginate(Bucket=bucket_id, Prefix=folder_name + header):
        all_images.extend(page.get('Contents', []))

    image_keys = sorted([obj['Key'] for obj in all_images])

    available_keys = []
    for i in range(num_previous_images + 1):
        index = -1 - i * image_delta_t-18915#aqui
        if abs(index) <= len(image_keys):
            available_keys.append(image_keys[index])

    # Descargar y procesar imágenes
    processed_images = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        results = list(tqdm(executor.map(download_and_process_image, available_keys), total=len(available_keys)))

    for key, proc_img in results:
        if proc_img is not None:
            processed_images[key] = proc_img

    return processed_images





image_dict = get_latest_images_from_s3(num_previous_images=2, image_delta_t=5)
# image_dict contains 3 entries spaced 15 minutes apart


keys=list(image_dict.keys())

def extract_timestamp_from_image(image_key):
    """
    Extracts the timestamp from the image key (of the form 'All_sky_camera/meanYYYY-MM-DD HH:MM:SS.ssssss.png'),
    and localizes it to the Europe/Zurich timezone.
    """
    timestamp_str = image_key.split('/')[1][4:-4]  # Extract timestamp string
    timestamp = pd.to_datetime(timestamp_str)
    localized_timestamp = timestamp.tz_localize('Europe/Zurich')
    localized_timestamp=localized_timestamp.tz_convert('UTC')
    return localized_timestamp



timestamp=extract_timestamp_from_image(keys[0])


Hdelta_t=16
delta_t=15



future_timestamps = [timestamp + i * timedelta(minutes=delta_t) for i in range(Hdelta_t + 1)]




#cs vector from timestamps
cs=batiment.get_clearsky(pd.DatetimeIndex(future_timestamps)).ghi.values


class InfluxDBQueryClient:
    def __init__(self, ip, port, token, org):
        self.url = f"http://{ip}:{port}"
        self.token = token
        self.org = org
        self.timeout=5e4
        self.client = InfluxDBClient(url=self.url, token=self.token, org=self.org,timeout=self.timeout)

    def query_measured_ghi(self, start_time, end_time, resolution):
        """
        Query the measured GHI from the InfluxDB database
        :param start_time: datetime.datetime
        :param end_time: datetime.datetime
        :param resolution: str
        :return: pd.DataFrame
        """
        bucket = "microgrid_ST"

        # Convert datetime to Unix timestamp
        t0 = round(start_time.timestamp())
        tf = round(end_time.timestamp())

        # Prepare the Flux query
        flux_query = f"""
        from(bucket: "{bucket}")
        |> range(start: {t0}, stop: {tf})
          |> filter(fn: (r) => r["_measurement"] == "microgrid")
          |> filter(fn: (r) => r["Resource"] == "meteobox_roof")
          |> filter(fn: (r) => r["_field"] == "GHI")
          |> aggregateWindow(every: {resolution}, fn: mean, createEmpty: false)
          |> yield(name: "mean")
          |> pivot(rowKey:["_time"], columnKey: ["Resource"], valueColumn: "_value")
        """

        # Query the data
        query_api = self.client.query_api()
        measured_GHI_api15 = query_api.query_data_frame(org=self.org, query=flux_query)
        return measured_GHI_api15[0]

    def query_ghi_DA_forecast(self, start_time, end_time, resolution):
        """
        Query the forecast GHI from solcast day ahead from the InfluxDB database
        :param start_time: datetime.datetime
        :param end_time: datetime.datetime
        :param resolution: str
        :return: pd.DataFrame
        """
        bucket = "logging"

        # Convert datetime to Unix timestamp
        t0 = round(start_time.timestamp())
        tf = round(end_time.timestamp())

        # Prepare the Flux query
        flux_query = f"""
        from(bucket: "{bucket}")
        |> range(start: {t0}, stop: {tf})
          |> filter(fn: (r) => r["_measurement"] == "WAday1")
          |> filter(fn: (r) => r["_field"] == "value")
          |> filter(fn: (r) => r["solcast_weekahead"] == "GHI" or r["solcast_weekahead"] == "GHI10" or r["solcast_weekahead"] == "GHI90" or r["solcast_weekahead"] == "airT")
          |> aggregateWindow(every: {resolution}, fn: mean, createEmpty: false)
          |> yield(name: "mean")
          |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
        """

        # Query the data
        query_api = self.client.query_api()
        ghi_DA_forecast = query_api.query_data_frame(org=self.org, query=flux_query)
        ghi_DA_forecast = ghi_DA_forecast.pivot(index='_time', columns='solcast_weekahead', values='_value').reset_index()
        ghi_DA_forecast = ghi_DA_forecast.rename(columns={"_time":'time'}).set_index('time')
        ghi_DA_forecast.columns.name = None
        return ghi_DA_forecast
    
    
    def query_nissan_leaf_voltage(self, start_time, end_time, resolution):
            """
            Query the CellVoltage of NissanLeaf from the InfluxDB database
            :param start_time: datetime.datetime
            :param end_time: datetime.datetime
            :param resolution: str
            :return: pd.DataFrame
            """
            bucket = "microgrid_ST"
            t0 = round(start_time.timestamp())
            tf = round(end_time.timestamp())

            flux_query = f"""
            from(bucket: "microgrid_ST")
            |> range(start: {t0}, stop: {tf})
            |> filter(fn: (r) => r["_measurement"] == "microgrid")
            |> filter(fn: (r) => r["Resource"] == "NissanLeaf")
            |> filter(fn: (r) => r["_field"] == "CellVoltage")
            |> aggregateWindow(every: {resolution}, fn: mean, createEmpty: false)
            |> yield(name: "mean")
            """

            query_api = self.client.query_api()
            nissan_leaf_voltage = query_api.query_data_frame(org=self.org, query=flux_query)
            nissan_leaf_voltage = nissan_leaf_voltage.pivot(index='_time', columns='Cell', values='_value').reset_index()
            return nissan_leaf_voltage 


    def pull_solcast_forecast(self, forecast_time):
        """
        Pull Solcast forecast data for GHI from InfluxDB.
        :param forecast_time: datetime.datetime, start of the query range
        :return: pd.DataFrame
        """
        to = forecast_time - timedelta(minutes=14)
        # Convert datetime to Unix timestamp
        tf = round(forecast_time.timestamp())
        t0 = round(to.timestamp())

        bucket = "Forecasting_ST"

        # Prepare the Flux query
        flux_query = f"""
        from(bucket: "{bucket}")
        |> range(start: {t0}, stop: {tf})
        |> filter(fn: (r) => r["type"] == "Solcast")
        |> filter(fn: (r) => r["_field"] == "ghi")
        |> pivot(rowKey:["_time"], columnKey: ["prediction_time"], valueColumn: "_value")
        |> yield(name: "mean")
        """
        # Query the data
        query_api = self.client.query_api()
        solcast_forecast = query_api.query_data_frame(org=self.org, query=flux_query)

        return solcast_forecast
    
    def pull_solcast_forecast_bulk(self, start_time, end_time):
        """
        Pull Solcast forecast data for GHI from InfluxDB over a time range.
        :param start_time: datetime.datetime
        :param end_time: datetime.datetime
        :return: pd.DataFrame
        """
        bucket = "Forecasting_ST"
        t0 = round(start_time.timestamp())
        tf = round(end_time.timestamp())
    
        flux_query = f"""
        from(bucket: "{bucket}")
        |> range(start: {t0}, stop: {tf})
        |> filter(fn: (r) => r["type"] == "Solcast")
        |> filter(fn: (r) => r["_field"] == "ghi")
        |> pivot(rowKey:["_time"], columnKey: ["prediction_time"], valueColumn: "_value")
        |> yield(name: "mean")
        """
    
        query_api = self.client.query_api()
        try:
            df = query_api.query_data_frame(org=self.org, query=flux_query)
        except Exception as e:
            print("Error al hacer la consulta:", e)
            raise

        #df = query_api.query_data_frame(org=self.org, query=flux_query)
        return df
    def pull_solcast_forecast_bulk2(self, start_time, end_time):
        bucket = "Forecasting_ST"
        t0 = round(start_time.timestamp())
        tf = round(end_time.timestamp())
    
        flux_query = f"""
        from(bucket: "{bucket}")
        |> range(start: {t0}, stop: {tf})
        |> filter(fn: (r) => r["type"] == "Solcast")
        |> filter(fn: (r) => r["_field"] == "ghi" or r["_field"] == "ghi10" or r["_field"] == "ghi90")
        |> pivot(rowKey:["_time"], columnKey: ["prediction_time", "_field"], valueColumn: "_value")
        |> yield(name: "mean")
        """
    
        query_api = self.client.query_api()
        try:
            df = query_api.query_data_frame(org=self.org, query=flux_query)
        except Exception as e:
            print("Error al hacer la consulta:", e)
            raise
    
        return df

client = InfluxDBQueryClient(
        ip='***',
        port=***,
        token="***",
        org="***",
    )
max_attempts = 5  # Set your maximum number of retries
attempts = 0

while attempts < max_attempts:
    try:
        ghi_df = client.query_measured_ghi(future_timestamps[0] - timedelta(minutes=1), future_timestamps[0], '5m')
        if ghi_df is not None and not ghi_df.empty:
            value = ghi_df['_value'].iloc[0]
            break
        print('1')  # Optional debug message
    except Exception:
        print('2')  # Optional error message
    attempts += 1
    time.sleep(2)
else:
    raise RuntimeError("Maximum number of attempts reached without success.")


ghi_df


solcast=client.pull_solcast_forecast_bulk2(timestamp-timedelta(minutes=15), timestamp)
solcast_last = solcast.iloc[[-1]].reset_index(drop=True)


lead_minutes = (np.arange(1, Hdelta_t+1) * 15.0).tolist()
mean_cols = [f"{m}_ghi" for m in lead_minutes]
q10_cols  = [f"{m}_ghi10" for m in lead_minutes]
q90_cols  = [f"{m}_ghi90" for m in lead_minutes]

solcast_means=solcast_last[mean_cols]
solcast_q10=solcast_last[q10_cols]
solcast_q90=solcast_last[q90_cols]


def create_model_quantiles_with_ghi_and_solcast(
    num_previous_images,
    Hdelta_t,
    dropout_rate=0.1,
    l2_lambda=1e-5,
    lambda_mean_loss=1.0,
    lambda_quantile_loss=150.0,
    lambda_width_loss=5
):
    H = Hdelta_t + 1

    # --- inputs ---
    image_input = layers.Input(shape=(num_previous_images + 1, 250, 250, 3), name='image_sequence')
    clear_sky_input = layers.Input(shape=(H,), name='clear_sky_input')
    ghi_now_input = layers.Input(shape=(1,), name='ghi_now_input')
    solcast_input = layers.Input(shape=(3, Hdelta_t), name='solcast_input')  # [3, Hdelta_t]

    # --- deeper 3D-CNN backbone ---
    x = layers.Conv3D(32, (3, 3, 3), padding='same', activation='relu')(image_input)
    x = layers.Conv3D(32, (3, 3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling3D((1, 2, 2))(x)

    x = layers.Conv3D(64, (3, 3, 3), padding='same', activation='relu')(x)
    x = layers.Conv3D(64, (3, 3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling3D((1, 2, 2))(x)

    # --- stacked ConvLSTM2D ---
    x = layers.ConvLSTM2D(64, (3, 3), padding='same', return_sequences=True, activation='tanh')(x)
    x = layers.ConvLSTM2D(64, (3, 3), padding='same', return_sequences=False, activation='tanh')(x)
    x = layers.LayerNormalization()(x)
    x = layers.SpatialDropout2D(dropout_rate)(x)

    # --- spatial pooling ---
    x = layers.GlobalAveragePooling2D()(x)

    # --- deeper clear-sky feature embedding ---
    cs = layers.Dense(128, activation='relu')(clear_sky_input)
    cs = layers.Dropout(dropout_rate)(cs)
    cs = layers.Dense(64, activation='relu')(cs)
    cs = layers.Dense(32, activation='relu')(cs)

    # --- solcast embedding ---
    s = layers.Flatten()(solcast_input)  # flatten [3, Hdelta_t]
    s = layers.Dense(64, activation='relu')(s)
    s = layers.Dropout(dropout_rate)(s)
    s = layers.Dense(32, activation='relu')(s)

    # --- concatenate all features ---
    x = layers.Concatenate()([x, cs, ghi_now_input, s])

    # --- MLP head with residual connection ---
    mlp_input = x
    x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda))(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(128, activation='relu')(x)

    mlp_proj = layers.Dense(128)(mlp_input)  # Project mlp_input to match x
    x = layers.Add()([x, mlp_proj])  # Residual connection

    x = layers.Dense(64, activation='relu')(x)

    # --- output: [H, 3] for [q10, mean, q90] per horizon ---
    head = layers.Dense(3 * H, activation='linear')(x)
    output = layers.Reshape((H, 3), name='quantiles')(head)

    model = Model(
        inputs=[image_input, clear_sky_input, ghi_now_input, solcast_input],
        outputs=output
    )

    # --- losses and optimizer ---
    weighted_mse_metric = weighted_compressed_mse(
        H, initial_weight=1.0, second_weight=1.0, decay=0.5
    )
    weighted_quantile_metric = weighted_quantile_loss(
        H, initial_weight=1.0, second_weight=1.0, decay=0.5
    )

    initial_lr = 3e-4
    drop_every_x_steps = 210 * 5
    lr_schedule = StepDecay(initial_lr, drop_every_x_steps)
    optimizer = tf.keras.optimizers.AdamW(learning_rate=lr_schedule)

    loss_fn = combined_quantile_loss_factory(
        H, lambda_mean=lambda_mean_loss, lambda_q=lambda_quantile_loss, lambda_width=lambda_width_loss
    )

    def mean_prediction_interval_width(y_true, y_pred):
        q10, _, q90 = tf.unstack(y_pred, axis=-1)
        width = q90 - q10
        return tf.reduce_mean(width)

    model.compile(
        optimizer=optimizer,
        loss=loss_fn,
        metrics=[
            mean_channel_mae,
            weighted_mse_metric,
            weighted_quantile_metric,
            mean_prediction_interval_width
        ]
    )

    #model.summary()
    return model

class StepDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, drop_every_x_steps):
        self.initial_lr = initial_lr
        self.drop_every_x_steps = drop_every_x_steps

    def __call__(self, step):
        factor = tf.math.floor(step / self.drop_every_x_steps)
        return self.initial_lr * tf.math.pow(0.3, factor)
def combined_quantile_loss_factory(
    H, lambda_mean=1.0, lambda_q=1.0, lambda_width=0.0,
    initial_weight=1.0, second_weight=1.0, decay=0.7):
    """
    Returns a loss(y_true, y_pred) that:
      - Combines MSE, quantile loss, and optional interval width penalty.
      - Applies initial_weight to horizon 0,
      - Applies second_weight * decay^(i-1) to horizons i >= 1.
    """
    def loss_fn(y_true, y_pred):
        q10, mu, q90 = tf.unstack(y_pred, axis=-1)  # [batch, H]

        # Per-horizon losses
        mse_h    = tf.reduce_mean(tf.square((y_true - mu) * 30), axis=0)
        pin10_h  = tf.reduce_mean(quantile_loss(0.1, y_true, q10), axis=0)
        pin90_h  = tf.reduce_mean(quantile_loss(0.9, y_true, q90), axis=0)
        width_h  = tf.reduce_mean(q90 - q10, axis=0)

        # Construct weights: [initial_weight, second_weight * decay^0, ..., decay^{H-2}]
        decay_factors = tf.pow(decay, tf.cast(tf.range(H - 1), tf.float32))  # [H-1]
        remaining_weights = second_weight * decay_factors
        weights = tf.concat([[initial_weight], remaining_weights], axis=0)  # shape [H]

        # Weighted losses
        weighted_mse = tf.reduce_sum(weights * mse_h)
        weighted_pinball = tf.reduce_sum(weights * (pin10_h + pin90_h))
        weighted_width = tf.reduce_sum(weights * width_h)

        total_loss = (
            lambda_mean * weighted_mse +
            lambda_q * weighted_pinball +
            lambda_width * weighted_width
        )
        normalization = tf.reduce_sum(weights)
        return total_loss / normalization

    return loss_fn
# --- 3) custom MAE that only looks at the mean channel ---
def mean_channel_mae(y_true, y_pred):
    """
    y_pred[...,1] is the mean prediction.
    """
    mean_pred = y_pred[..., 1]
    return tf.reduce_mean(tf.abs(y_true - mean_pred))
def weighted_compressed_mse(H, initial_weight=1.0, second_weight=1.0, decay=0.5):
    """
    Returns a metric function that computes a weighted, scaled MSE across horizons.
    Weights:
      - Horizon 0 → initial_weight
      - Horizon i ≥ 1 → second_weight * decay^(i-1)
    """
    def Mean_metric(y_true, y_pred):
        mu = y_pred[..., 1]  # extract predicted mean, shape [batch, H]
        error = (y_true - mu) * 30
        mse_per_horizon = tf.reduce_mean(tf.square(erprintror), axis=0)  # shape [H]

        # Compute weights
        decay_factors = tf.pow(decay, tf.cast(tf.range(H - 1), tf.float32))
        remaining_weights = second_weight * decay_factors
        weights = tf.concat([[initial_weight], remaining_weights], axis=0)  # shape [H]

        weighted_mse = tf.reduce_sum(weights * mse_per_horizon)
        normalization = tf.reduce_sum(weights)
        return weighted_mse / normalization

    return Mean_metric
def weighted_quantile_loss(H, initial_weight=1.0, second_weight=1.0, decay=0.5):
    """
    Returns a metric function that computes the weighted average quantile loss
    (pinball loss) over q10 and q90 across horizons.
    """
    def Q_metric(y_true, y_pred):
        q10 = y_pred[..., 0]
        q90 = y_pred[..., 2]

        # Pinball loss per horizon
        pin10_h = tf.reduce_mean(quantile_loss(0.1, y_true, q10), axis=0)
        pin90_h = tf.reduce_mean(quantile_loss(0.9, y_true, q90), axis=0)

        # Construct weights
        decay_factors = tf.pow(decay, tf.cast(tf.range(H - 1), tf.float32))
        remaining_weights = second_weight * decay_factors
        weights = tf.concat([[initial_weight], remaining_weights], axis=0)  # shape [H]

        weighted_pinball = tf.reduce_sum(weights * (pin10_h + pin90_h))
        normalization = tf.reduce_sum(weights)
        return 50*weighted_pinball / normalization

    return Q_metric


num_previous_images=2
Hdelta_t=16

from tensorflow.keras import layers, Model, regularizers
print('a')
model=create_model_quantiles_with_ghi_and_solcast(
    num_previous_images,
    Hdelta_t)
print('b')
model.load_weights('***') 

print('weights laoded')

#model = tf.keras.models.load_model('with_solcast_model.keras')


images_array = np.stack([image_dict[k].astype(np.float32) / 255.0 for k in keys]) 

timestamp_day = pd.Timestamp(timestamp).normalize()

day_times = pd.date_range(
    start=timestamp_day,
    end=timestamp_day + pd.Timedelta(days=1) - pd.Timedelta(seconds=1),
    freq='15min'
)

cs_for_max = batiment.get_clearsky(day_times).ghi
max_cs = cs_for_max.max()

print("Max clear-sky GHI on", timestamp_day.date(), "=", max_cs)

solcast_q10

solcast_q10 = solcast_q10.to_numpy().flatten()
solcast_means = solcast_means.to_numpy().flatten()
solcast_q90 = solcast_q90.to_numpy().flatten()


images_input = np.expand_dims(images_array, axis=0)          # (1, 3, 250, 250, 3)
clear_sky_input = np.expand_dims(cs, axis=0)/max_cs    # (1, 17)
ghi_now_input = np.expand_dims(value, axis=0)/max_cs       # (1, 1)
solcast_input=np.expand_dims(np.stack([solcast_q10, solcast_means, solcast_q90], axis=0),axis=0)/max_cs               #(1,3,Hdelta_t)
# Diccionario de inputs
inputs = {
    'image_sequence': images_input,
    'clear_sky_input': clear_sky_input,
    'ghi_now_input': ghi_now_input,
    'solcast_input': solcast_input
}

# Predicción
prediction = model.predict(inputs)


print(prediction*max_cs)

real_forecast=prediction*max_cs

bucket = "***"
org = "***"
token = "***"
IP = '***'
PORT = ***
url = f"http://{IP}:{PORT}"
client = InfluxDBClient(url=url, token=token, org=org)

is_healthy = client.ping()
print("InfluxDB connection is healthy:", is_healthy)

time

import pandas as pd
import numpy as np
import datetime
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

def log_dwd_influx( future_times,Hdelta_t,delta_t,real_forecast,location='EPFL'):
    initial_time=future_times[0]

    # InfluxDB connection details
    bucket = "***"
    org = "***"
    token = "***"
    IP = '***'
    PORT = ***
    url = f"http://{IP}:{PORT}"
    client = InfluxDBClient(url=url, token=token, org=org)
    write_api = client.write_api(write_options=SYNCHRONOUS)


    # Get the time of the forecast
    from zoneinfo import ZoneInfo
    # specify the timezone of the prediction time, alreasy has
    # Convert to nanoseconds since the UNIX epoch
    timestamp_ns = int(initial_time.timestamp() * 1e9)
    print(timestamp_ns)
    # Get the fields we want to log
   
    fields = ["GHI_10Q", "GHI_AVG", "GHI_90Q"]
    
    # Create all the irradiance points to send to InfluxDB
    points_irr = []
    future_times=np.arange(0,(Hdelta_t+1)*delta_t,delta_t)
    for t in range(len(future_times)):
        for f in range(len(fields)):
            prediction_time = future_times[t]
            value = real_forecast[t][f]
            print(value, str(prediction_time))
            point_irr = Point("Forecast")\
                .tag("type", "All_sky_camera")\
                .tag("prediction_time", str(prediction_time))\
                .tag("location", location)\
                .field(fields[f], value)\
                .time(timestamp_ns )  # Use timestamp from array
    
            points_irr.append(point_irr)
    #print(points_irr)
    # Write log_dwd_influx(forecast_time,  prediction_times,location='EPFL')to InfluxDB
    write_api.write(bucket=bucket, org=org, record=points_irr)

  

log_dwd_influx( future_timestamps,Hdelta_t,delta_t,real_forecast[0],location='EPFL')
print(real_forecast)

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  6.51it/s]


a
b
weights laoded
Max clear-sky GHI on 2025-05-30 = 874.3637293802624
1/1 [==============================] - 0s 268ms/step
[[[586.18787 650.4661  661.3385 ]
  [496.1683  502.7906  573.24054]
  [485.06497 483.52374 439.4149 ]
  [424.19778 469.70602 535.37195]
  [419.89423 446.34555 429.36725]
  [477.57388 466.77856 495.04813]
  [403.11954 383.32886 516.2467 ]
  [387.20074 498.72586 455.66193]
  [393.55914 450.21808 548.3646 ]
  [336.54318 486.9724  445.3326 ]
  [284.6566  432.04446 495.43982]
  [296.68658 396.915   491.57385]
  [313.0871  343.2408  506.67694]
  [237.14198 350.07175 385.36206]
  [246.65596 400.09018 511.0394 ]
  [277.89508 301.26877 541.0569 ]
  [170.12335 320.14862 443.42984]]]
InfluxDB connection is healthy: True
1748595064271418112
586.18787 0
650.4661 0
661.3385 0
496.1683 15
502.7906 15
573.24054 15
485.06497 30
483.52374 30
439.4149 30
424.19778 45
469.70602 45
535.37195 45
419.89423 60
446.34555 60
429.36725 60
477.57388 75
466.77856 75
495.04813 75
403.11954 90


In [ ]:
max_cs

In [6]:
ghi_now_input*max_cs

array([420.33536927])

In [31]:
keys

['All_sky_camera/mean2025-05-30 10:51:04.271418.png',
 'All_sky_camera/mean2025-05-30 10:46:05.187168.png',
 'All_sky_camera/mean2025-05-30 10:41:05.021083.png']

In [32]:
prediction

array([[[0.67041653, 0.7439308 , 0.7563654 ],
        [0.56746215, 0.575036  , 0.6556088 ],
        [0.5547634 , 0.5530007 , 0.5025539 ],
        [0.48515028, 0.53719753, 0.6122989 ],
        [0.48022833, 0.5104804 , 0.49106252],
        [0.5461959 , 0.5338494 , 0.566181  ],
        [0.4610433 , 0.43840894, 0.59042555],
        [0.44283718, 0.5703872 , 0.52113545],
        [0.45010918, 0.5149094 , 0.62715846],
        [0.38490066, 0.5569449 , 0.5093219 ],
        [0.32555854, 0.4941244 , 0.566629  ],
        [0.3393171 , 0.45394725, 0.5622075 ],
        [0.35807422, 0.3925607 , 0.57948077],
        [0.27121663, 0.40037316, 0.4407343 ],
        [0.28209767, 0.45757866, 0.58447003],
        [0.3178255 , 0.34455773, 0.6188007 ],
        [0.19456817, 0.3661504 , 0.50714576]]], dtype=float32)